In [25]:
import torch 

import torch.cuda as cuda

x = torch.randn((256,1024)).cuda()
w = torch.nn.Linear(1024,1024,bias=False,device='cuda')
print(f"memory: 4M model + 1M input = {cuda.memory_allocated()//2**20}M")
for i in range(5):
    x = w(x)
    print(f"reserve w(x)_{i} activations, 1M")
    print(f"{cuda.memory_allocated()//2**20}M")

del x

memory: 4M model + 1M input = 5M
reserve w(x)_0 activations, 1M
6M
reserve w(x)_1 activations, 1M
7M
reserve w(x)_2 activations, 1M
8M
reserve w(x)_3 activations, 1M
9M
reserve w(x)_4 activations, 1M
10M


In [ ]:

import torch

import torch.cuda as cuda

x = torch.randn((256,1024)).cuda()
w = torch.nn.Linear(1024,1024,bias=False,device='cuda')
print(f"memory: 4M model + 1M input = {cuda.memory_allocated()//2**20}M")
for i in range(5):
    x = w(x)
    print(f"reserve w(x)_{i} activations, 1M")
    print(f"{cuda.memory_allocated()//2**20}M")

del x

In [1]:
import torch

import torch.cuda as cuda
small_dim , large_dim = 1024,4096
x = torch.randn((256,4096)).cuda()
w1 = torch.nn.Linear(small_dim,small_dim,bias=False,device='cuda')
w2 = torch.nn.Linear(large_dim,large_dim,bias=False,device='cuda')
up_w = torch.nn.Linear(small_dim,large_dim,bias=False,device='cuda')
down_w = torch.nn.Linear(large_dim,small_dim,bias=False,device='cuda')
print(f"memory: 4M model + 1M input = {cuda.memory_allocated()//2**20}M")

# freeze the large model
# w1.weight.requires_grad_(False)
w2.weight.requires_grad_(False)

hidden_states_small = down_w(x)
hidden_states_large = x
for i in range(5):
    print(f"{cuda.memory_allocated()//2**20}M")
    hidden_states_large = w2(hidden_states_large)
    print(f"{cuda.memory_allocated()//2**20}M")
    # infusion from large
    hidden_states_small = .5 * down_w(hidden_states_large) + .5 * hidden_states_small
    # hidden_states_small = .5 * hidden_states_small + .5 * hidden_states_small
    hidden_states_small = w1(hidden_states_small)
    # infusion from small
    hidden_states_large = .5 * up_w(hidden_states_small) + .5 * hidden_states_large
    
    # print(f"reserve w(x)_{i} activations, 1M")
    # print(f"{cuda.memory_allocated()//2**20}M")

del x

/home/czz/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


memory: 4M model + 1M input = 104M
105M
109M
114M
114M
120M
120M
126M
126M
132M
132M
